In [ ]:
# Jason Qin
# 05/05/21

# Organize VDJdb TCR-antigen data

Organize data by:
1) TCR chain
2) Species
3) Unique antigens

In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from scipy.spatial.distance import cdist, hamming
import scipy.stats
import sklearn.metrics

from tape.datasets import FastaDataset, pad_sequences
from tape.tokenizers import TAPETokenizer

In [16]:
# relevant data files
data_dir = '/group/ag_cmb/scratch/jqin/projects/tape/data/tcr'
data_file = data_dir + '/full_data.tsv'
data = pd.read_csv(data_file, sep='\t')

### Organize Sequences by Species

Find all CDR3 sequences for each species

In [29]:
species = ['HomoSapiens', 'MusMusculus', 'MacacaMulatta']
species_cdr3_seq = {} # dict mapping species to all CDR3 sequences that come from it
for sp in species:
    species_cdr3_seq[sp] = set(data[data['Species'] == sp]['CDR3'].tolist())

In [30]:
# write the genes to file
for sp in species:
    with open('{}/{}_cdr3.fasta'.format(data_dir, sp), 'w') as f:
        for i, seq in enumerate(species_cdr3_seq[sp]):
            f.write('>SEQ_{}\n'.format(i))
            f.write('{}\n'.format(seq))

In [32]:
human_cdr3 = species_cdr3_seq['HomoSapiens']
macaque_cdr3 = species_cdr3_seq['MacacaMulatta']
mouse_cdr3 = species_cdr3_seq['MusMusculus']

In [33]:
# are there any CDR3 sequences common between the three species?
print('Human and macaque CDR3 overlap: {}'.format(human_cdr3.intersection(macaque_cdr3)))
print('Human and mouse CDR3 overlap: {}'.format(human_cdr3.intersection(mouse_cdr3)))
print('Mouse and macaque CDR3 overlap: {}'.format(mouse_cdr3.intersection(macaque_cdr3)))

Human and macaque CDR3 overlap: set()
Human and mouse CDR3 overlap: {'CAVSEGNYQLIW', 'CASSEQGYEQYF', 'CASSDAGTGDYEQYF', 'CASSLDRGEQYF', 'CASSLSGYEQYF', 'CASSPGQGYEQYF', 'CAALDSNYQLIW', 'CAVDSNYQLIW', 'CALIGGSNYKLTF', 'CASRQGYEQYF', 'CASSLGDEQYF', 'CAALNSGGSNYKLTF', 'CVVQDSNYQLIW', 'CASSFGREQYF', 'CAVGSNYQLIW', 'CASSESYEQYF', 'CAMRHSNYQLIW', 'CASSLRGAYEQYF', 'CAYSGGSNYKLTF', 'CASRDTYEQYF', 'CAVRPGNTGKLIF', 'CASSYSYEQYF', 'CASSLGGYEQYF', 'CAVSMDSNYQLIW', 'CAASGNYQLIW', 'CAVNSNYQLIW', 'CAASASGGSNYKLTF', 'CAASMDSNYQLIW', 'CASSLSGAYEQYF', 'CAMREGSGGSNYKLTF', 'CAVSLDSNYQLIW', 'CASSEGGPYEQYF', 'CATGSNYQLIW', 'CAVMDSNYQLIW', 'CAVSGGDSNYQLIW', 'CASSSGGEQYF', 'CASSLGLYEQYF', 'CASSFDSYEQYF'}
Mouse and macaque CDR3 overlap: set()


In [39]:
print('Number of CDR3 sequences shared between human and mouse: {}'.format(len(human_cdr3.intersection(mouse_cdr3))))
print('Number of unique mouse CDR3 sequences: {}'.format(len(mouse_cdr3)))
print('Number of unique human CDR3 sequences: {}'.format(len(human_cdr3)))

Number of CDR3 sequences shared between human and mouse: 38
Number of unique mouse CDR3 sequences: 3197
Number of unique human CDR3 sequences: 52897


Find all antigen sequences for each species

In [19]:
# want to do: do any species have the same antigens? If so, may need to
# be careful about separating datasets down the line
species_antigen_seq = {}
for sp in species:
    species_antigen_seq[sp] = set(data[data['Species'] == sp]['Epitope'].tolist())

In [47]:
# write the antigens to file
for sp in species:
    with open('{}/{}_antigen.fasta'.format(data_dir, sp), 'w') as f:
        for i, seq in enumerate(species_antigen_seq[sp]):
            f.write('>SEQ_{}\n'.format(i))
            f.write('{}\n'.format(seq))

In [20]:
human_antigens = species_antigen_seq['HomoSapiens']
macaque_antigens = species_antigen_seq['MacacaMulatta']
mouse_antigens = species_antigen_seq['MusMusculus']

In [25]:
# are any antigens common between the three species?
print('Human and macaque antigen overlap: {}'.format(human_antigens.intersection(macaque_antigens)))
print('Human and mouse antigen overlap: {}'.format(human_antigens.intersection(mouse_antigens)))
print('Mouse and macaque antigen overlap: {}'.format(mouse_antigens.intersection(macaque_antigens)))

Human and macaque antigen overlap: set()
Human and mouse antigen overlap: set()
Mouse and macaque antigen overlap: set()


Takeaways:
- All antigens are unique between different species
- Most TCR sequences are unique between different species
- But: some TCR sequences are shared between mouse and human, despite the fact that no antigen sequences are shared. This means that the same TCR sequence binds to different antigens depending on the species

### Organize Sequence by TRA vs TRB Chain

In [28]:
chains = ['TRA', 'TRB']
chain_cdr3_seq = {} # dict mapping TRA/TRB to all CDR3 sequences that come from it
for chain in chains:
    chain_cdr3_seq[chain] = set(data[data['Gene'] == chain]['CDR3'].tolist())

In [45]:
# write the genes to file
for chain in chains:
    with open('{}/{}_cdr3.fasta'.format(data_dir, chain), 'w') as f:
        for i, seq in enumerate(chain_cdr3_seq[chain]):
            f.write('>SEQ_{}\n'.format(i))
            f.write('{}\n'.format(seq))

In [40]:
# are there any CDR3 sequences common between the two genes?
tra_cdr3 = chain_cdr3_seq['TRA']
trb_cdr3 = chain_cdr3_seq['TRB']

In [44]:
print('TRA and TRB CDR3 overlap: {}'.format(tra_cdr3.intersection(trb_cdr3)))
print('TRA CDR3 sequences: {}'.format(len(tra_cdr3)))
print('TRB CDR3 sequences: {}'.format(len(trb_cdr3)))

TRA and TRB CDR3 overlap: {'CASSDSRGTEAFF', 'CASSSVNEQYF'}
TRA CDR3 sequences: 22352
TRB CDR3 sequences: 34900


### Organize data by unique antigen

In [49]:
# keep track of counts and locations of each antigen
antigens = data['Epitope']
antigen_loc_dict = {}
antigen_count_dict = {}
for i, antigen in enumerate(antigens):
    if antigen not in antigen_loc_dict.keys():
        antigen_loc_dict[antigen] = [i]
        antigen_count_dict[antigen] = 0
    else:
        antigen_loc_dict[antigen] += [i]
        antigen_count_dict[antigen] += 1

In [52]:
# find all antigens with same copy number in data
count_antigen_dict = {}
for antigen, count in antigen_count_dict.items():
    if count not in count_antigen_dict.keys():
        count_antigen_dict[count] = [antigen]
    else:
        count_antigen_dict[count] += [antigen]

In [89]:
# split data into 90/5/5 train/valid/test
total_counts = data.shape[0]
data_counts = {'train': int(total_counts * 0.9), 
               'valid': int(total_counts * 0.05), 
               'test': total_counts - train_counts - valid_counts}

In [92]:
# keep track of antigen sequences for train/valid/test sets
antigen_data_split_lists = {'train': [],
                            'valid': [],
                            'test': []}
current_counts = {'train': 0, 
                  'valid': 0, 
                  'test': 0}

for count, antigens in count_antigen_dict.items():
    i = 0
    for antigen in antigens:
        if (i%3 == 0) and (current_counts['train'] < data_counts['train']*0.9):
            antigen_data_split_lists['train'] += [antigen]
            current_counts['train'] += count
        elif (i%3 == 1) and (current_counts['valid'] < data_counts['valid']*0.9):
            antigen_data_split_lists['valid'] += [antigen]
            current_counts['valid'] += count
        else:
            if current_counts['test'] < data_counts['test']:
                antigen_data_split_lists['test'] += [antigen]
                current_counts['test'] += count
            else:
                antigen_data_split_lists['valid'] += [antigen]
                current_counts['valid'] += count
        i += 1

In [97]:
# find indices in data for antigens in train/valid/test sets
antigen_data_split_idx = {'train': [],
                          'valid': [],
                          'test': []}

for data_split, antigens in antigen_data_split_lists.items():
    for antigen in antigens:
        antigen_data_split_idx[data_split] += antigen_loc_dict[antigen]

In [101]:
# write antigens and CDR3 sequences to file
for data_split, idx in antigen_data_split_idx.items():
    antigen_seq = data['Epitope'][idx]
    with open('{}/antigen_unique_{}.fasta'.format(data_dir, data_split), 'w') as f:
        for seq in antigen_seq:
            f.write('>SEQ_{}\n'.format(i))
            f.write('{}\n'.format(seq))
    
    cdr3_seq = data['CDR3'][idx]
    with open('{}/cdr3_unique_{}.fasta'.format(data_dir, data_split), 'w') as f:
        for seq in cdr3_seq:
            f.write('>SEQ_{}\n'.format(i))
            f.write('{}\n'.format(seq))

In [98]:
data

,complex.id,Gene,CDR3,V,J,Species,MHC A,MHC B,MHC class,Epitope,Epitope gene,Epitope species,Reference,Method,Meta,CDR3fix,Score
0,1,TRA,CIVRAPGRADMRF,TRAV26-1*01,TRAJ43*01,HomoSapiens,HLA-B*08,B2M,MHCI,FLKEKGGL,Nef,HIV-1,PMID:15596521,"{""frequency"": """", ""identification"": ""tetramer-...","{""cell.subset"": ""CD8+"", ""clone.id"": """", ""donor...","{""cdr3"": ""CIVRAPGRADMRF"", ""cdr3_old"": ""CIVRAPG...",2
1,1,TRB,CASSYLPGQGDHYSNQPQHF,TRBV13*01,TRBJ1-5*01,HomoSapiens,HLA-B*08,B2M,MHCI,FLKEKGGL,Nef,HIV-1,PMID:15596521,"{""frequency"": """", ""identification"": ""tetramer-...","{""cell.subset"": ""CD8+"", ""clone.id"": """", ""donor...","{""cdr3"": ""CASSYLPGQGDHYSNQPQHF"", ""cdr3_old"": ""...",2
2,0,TRB,CASSFEAGQGFFSNQPQHF,TRBV13*01,TRBJ1-5*01,HomoSapiens,HLA-B*08,B2M,MHCI,FLKEKGGL,Nef,HIV-1,PMID:15596521,"{""frequency"": """", ""identification"": ""tetramer-...","{""cell.subset"": ""CD8+"", ""clone.id"": """", ""donor...","{""cdr3"": ""CASSFEAGQGFFSNQPQHF"", ""cdr3_old"": ""C...",2
3,2,TRA,CAVPSGAGSYQLTF,TRAV20*01,TRAJ28*01,HomoSapiens,HLA-B*08,B2M,MHCI,FLKEKGGL,Nef,HIV-1,PMID:15596521,"{""frequency"": """", ""identification"": ""tetramer-...","{""cell.subset"": ""CD8+"", ""clone.id"": """", ""donor...","{""cdr3"": ""CAVPSGAGSYQLTF"", ""cdr3_old"": ""CAVPSG...",2
4,2,TRB,CASSFEPGQGFYSNQPQHF,TRBV13*01,TRBJ1-5*01,HomoSapiens,HLA-B*08,B2M,MHCI,FLKEKGGL,Nef,HIV-1,PMID:15596521,"{""frequency"": """", ""identification"": ""tetramer-...","{""cell.subset"": ""CD8+"", ""clone.id"": """", ""donor...","{""cdr3"": ""CASSFEPGQGFYSNQPQHF"", ""cdr3_old"": ""C...",2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77671,24823,TRA,CAGQRATDSWGKLQF,TRAV25*01,TRAJ24*01,HomoSapiens,HLA-A*02:01,B2M,MHCI,LLWNGPMAV,NS4B,YFV,PMID:28934479,"{""frequency"": ""1/109"", ""identification"": ""tetr...","{""cell.subset"": """", ""clone.id"": """", ""donor.MHC...","{""cdr3"": ""CAGQRATDSWGKLQF"", ""cdr3_old"": ""CAGQR...",0
77672,24829,TRA,CAVDGQKLLF,TRAV12-2*01,NaN,HomoSapiens,HLA-A*02:01,B2M,MHCI,LLWNGPMAV,NS4B,YFV,PMID:28934479,"{""frequency"": ""1/109"", ""identification"": ""tetr...","{""cell.subset"": """", ""clone.id"": """", ""donor.MHC...","{""cdr3"": ""CAVDGQKLLF"", ""cdr3_old"": ""CAVDGQKLLF...",0
77673,24834,TRA,CAASQETSGSRLTF,TRAV13-1*01,NaN,HomoSapiens,HLA-A*02:01,B2M,MHCI,LLWNGPMAV,NS4B,YFV,PMID:28934479,"{""frequency"": ""1/109"", ""identification"": ""tetr...","{""cell.subset"": """", ""clone.id"": """", ""donor.MHC...","{""cdr3"": ""CAASQETSGSRLTF"", ""cdr3_old"": ""CAASQE...",0
77674,24848,TRA,CFHYGGATNKLIF,TRAV1-2*01,TRAJ32*01,HomoSapiens,HLA-A*02:01,B2M,MHCI,LLWNGPMAV,NS4B,YFV,PMID:28934479,"{""frequency"": ""1/109"", ""identification"": ""tetr...","{""cell.subset"": """", ""clone.id"": """", ""donor.MHC...","{""cdr3"": ""CFHYGGATNKLIF"", ""cdr3_old"": ""CFHYGGA...",0
